# Generics

As in java, classes in Kotlin may have type parameters

In [8]:
class Box<T>(t: T){
    var value = t
}

In general, to create an instance of such a class, we need to provide the type arguements

In [10]:
val box: Box<Int> = Box<Int>(1)

But if the parameters may be inferred, for example from the constructor arguements or by some other means, one is allowed to omit the type arguements:

In [11]:
val box = Box(1)

## Variance

One of the most tricky parts of Java's type system is wildcard types And Kotlin doesn't have any. Instead. it has two other things: **declaration-site variance and type projections**.


First, lets talk about why java need wildcards. First, generic types in java are **invariant**, meaning that `List<String>` is not subtype of `List<Object>`. Why so? If List was not **invariant**, it would have been no better than the java arrays, since the following code would have compiled amd caused an exception at runtime.

``` java
List<String> strs = new ArrayList<String>();
List<Object> objs = strs; // !!! The cause of the upcoming problem sits here. Java prohibits this!
objs.add(1); // Here we put an Integer into a list of Strings
String s = strs.get(0); // !!! ClassCastException: Cannot cast Integer to String 
```

So, Java prohibits such things in order to guarantee run-time safety. But this has some implications. For example, consider the `addAll()`method from Collection interface.What would be the intutive signature of the above method. It would something like this
    
``` java
interface Collection<E> ... {
  void addAll(Collection<E> items);
}
```

But we try to do a simple thing like this it will fail to compile

``` java
void copyAll(Collection<Object> to, Collection<String> from) {
  to.addAll(from);
  // !!! Would not compile with the naive declaration of addAll:
  // Collection<String> is not a subtype of Collection<Object>
}
```

Thats why the actual signature of the `addAll()` is this
``` java
interface Collection<E> ... {
  void addAll(Collection<? extends E> items);
}
```

<p>The <strong>wildcard type argument</strong> <code>? extends E</code> indicates that this method accepts a collection of objects of <code>E</code> <em>or some subtype of</em> <code>E</code>, not just <code>E</code> itself. 
This means that we can safely <strong>read</strong> <code>E</code>'s from items (elements of this collection are instances of a subclass of E), but <strong>cannot write</strong> to 
it since we do not know what objects comply to that unknown subtype of <code>E</code>. 
In return for this limitation, we have the desired behaviour: <code>Collection&lt;String&gt;</code> <em>is</em> a subtype of <code>Collection&lt;? extends Object&gt;</code>. 
In "clever words", the wildcard with an <strong>extends</strong>-bound (<strong>upper</strong> bound) makes the type <strong>covariant</strong>.</p>


<p>The key to understanding why this trick works is rather simple: if you can only <strong>take</strong> items from a collection, then using a collection of <code>String</code>s
and reading <code>Object</code>s from it is fine. Conversely, if you can only <em>put</em> items into the collection, it's OK to take a collection of
<code>Object</code>s and put <code>String</code>s into it: in Java we have <code>List&lt;? super String&gt;</code> a <strong>supertype</strong> of <code>List&lt;Object&gt;</code>.</p>


<p>The latter is called <strong>contravariance</strong>, and you can only call methods that take String as an argument on <code>List&lt;? super String&gt;</code> 
(e.g., you can call <code>add(String)</code> or <code>set(int, String)</code>), while 
if you call something that returns <code>T</code> in <code>List&lt;T&gt;</code>, you don't get a <code>String</code>, but an <code>Object</code>.</p>

